In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
data_orig = pd.read_csv('USGovtRevCollect_20041001_20231227.csv')

C:\Users\josep\AppData\Local\Temp\ipykernel_19160\2384280357.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = data_orig

df.isnull().sum()
df.drop_duplicates(inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81248 entries, 0 to 81247
Data columns (total 15 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Record Date                      81248 non-null  object 
 1   Electronic Category Description  81248 non-null  object 
 2   Channel Type Description         81248 non-null  object 
 3   Tax Category Description         81248 non-null  object 
 4   Net Collections Amount           81248 non-null  float64
 5   Electronic Category ID           81248 non-null  int64  
 6   Channel Type ID                  81248 non-null  int64  
 7   Tax Category ID                  81248 non-null  int64  
 8   Source Line Number               81248 non-null  int64  
 9   Fiscal Year                      81248 non-null  int64  
 10  Fiscal Quarter Number            81248 non-null  int64  
 11  Calendar Year                    81248 non-null  int64  
 12  Calendar Quarter N

In [3]:
df['Record Date'] = pd.to_datetime(df['Record Date'])
record_date = df['Record Date'].dt
record_year = record_date.year
record_month = record_date.month
record_day = record_date.day
record_quarter = record_date.quarter

year_match = (record_year == df['Calendar Year']).all()
month_match = (record_month == df['Calendar Month Number']).all()
day_match = (record_day == df['Calendar Day Number']).all()
quarter_match = (record_quarter == df['Calendar Quarter Number']).all()
print("year match =", year_match)
print("month match =", month_match)
print("day match =", day_match)
print("quarter match =", quarter_match)

year match = True
month match = True
day match = True
quarter match = True


There is 'Record Date' in the data, and there are also 'Calendar Year, Month, Day, Quarter' respectively.  
After checking, they are the same, so one of them can be removed.

In [4]:
df_sorted = df.sort_values(by='Record Date')
df_sorted['Date Diff'] = df_sorted['Record Date'].diff().dt.days
date_diff = df_sorted['Date Diff'].value_counts().sort_index()
print(date_diff)

Date Diff
0.0    76414
1.0     3779
2.0       50
3.0      869
4.0      135
Name: count, dtype: int64


By looking at the number of days between adjacent record dates,  
we found that the interval between most dates is 0 or 1  
and the largest interval is only 4 days. 
This shows that the dates are continuous and we don't need to remap them

In [5]:
fiscal_date = df.groupby(['Fiscal Year', 'Fiscal Quarter Number']).size()
fiscal_date_unstack = fiscal_date.unstack(fill_value=0)
print(fiscal_date_unstack)
fiscal_date_std = fiscal_date.std()
print("Standard deviation =", fiscal_date_std)

Fiscal Quarter Number     1     2     3     4
Fiscal Year                                  
2005                   1093  1096  1219  1293
2006                   1219  1237  1281  1266
2007                   1258  1292  1364  1341
2008                   1253  1160  1195  1202
2009                   1129  1098  1149  1170
2010                   1101  1081  1128  1139
2011                   1146  1148  1198  1250
2012                   1181  1154  1177  1144
2013                   1064  1087  1181  1165
2014                   1107   995  1029  1040
2015                    993   977  1024  1049
2016                   1001   988  1024  1038
2017                    975   979   992   974
2018                    926   947   996   973
2019                    943   927   984   947
2020                    881   868   849   914
2021                    862   827   886   886
2022                    876   846   859   873
2023                    822   862   876   905
2024                    869     0 

// comment:  The federal government's fiscal year runs from October 1 to September 30.  
Through observation, we found that the standard deviation of the statiscal numbers in different fiscal quarters is not large  
which shows that the number of records in different fiscals is relatively evenly distributed  
we can use this data for analysis without cleaning

In [6]:
df.groupby('Electronic Category Description').size()

Electronic Category Description
Electronic Settlement     12466
Fully Electronic - All    31609
Fully Electronic - FS     14129
Non-Electronic            23044
dtype: int64

In [7]:
df.groupby('Channel Type Description').size()

Channel Type Description
Bank                      12443
Internet                  13911
Mail                      20345
Other                        66
Over-the-Counter (OTC)    34483
dtype: int64

In [8]:
df.groupby('Tax Category Description').size()

Tax Category Description
IRS Non-Tax    18545
IRS Tax        24843
Non-Tax        37860
dtype: int64

There are a few categories in these three strings 'Electronic Category Description', 'Channel Type Description', and 'Tax Category Description'  
We don't know how it contributes to our goals for the time being  
so we first assume that they are independent and use IDs

In [9]:
df.groupby('Source Line Number').size()

Source Line Number
1     4834
2     4834
3     4834
4     4834
5     4834
6     4834
7     4834
8     4834
9     4834
10    4834
11    4834
12    4828
13    4790
14    4465
15    3947
16    3336
17    2427
18    2026
19    1201
20     657
21     303
22      92
23       2
dtype: int64

// comments: Indicates the row of the corresponding table where the data point can be found.  
This data seems not stronly related to our goals  
If we want fewer features, maybe ignore it for now

In [10]:
df_processed = df.copy()
min_date = df_processed['Record Date'].min()
df_processed['Record Date Discretized'] = (df_processed['Record Date'] - min_date).dt.days
df_processed.drop(['Record Date', 'Source Line Number', 'Electronic Category Description', 'Channel Type Description', 'Tax Category Description'], axis=1, inplace=True)
df_processed

,Net Collections Amount,Electronic Category ID,Channel Type ID,Tax Category ID,Fiscal Year,Fiscal Quarter Number,Calendar Year,Calendar Quarter Number,Calendar Month Number,Calendar Day Number,Record Date Discretized
0,7.269500e+02,4,2,1,2024,1,2023,4,12,27,7026
1,2.431048e+05,3,1,1,2024,1,2023,4,12,27,7026
2,9.395110e+06,4,1,1,2024,1,2023,4,12,27,7026
3,4.824105e+04,2,1,2,2024,1,2023,4,12,27,7026
4,2.905820e+03,1,5,3,2024,1,2023,4,12,27,7026
...,...,...,...,...,...,...,...,...,...,...,...
81243,4.838831e+09,1,5,3,2005,1,2004,4,10,1,0
81244,2.114361e+06,4,5,1,2005,1,2004,4,10,1,0
81245,1.183413e+07,1,3,1,2005,1,2004,4,10,1,0
81246,1.921165e+06,2,1,1,2005,1,2004,4,10,1,0


We do the following preprocessing:
1. Delete 'Source Line Number', 'Electronic Category Description', 'Channel Type Description', 'Tax Category Description'
2. Discretize the Record Date into an integer, because we guess that the target has a polynomial relationship with this number
3. Remove the string class and keep only the encoded integers
We guessed that fiscal quarter would be an interesting feature, so we kept the information  
For record date, we don’t know whether quarter will be related to goals, so we keep the year, month, day, quarter and discretized integers.  

But because they are not independent of each other, during actual analysis, there are several options:
1. Discretized integers
2. year, month, day
3. year, quarter
4. Others

In [11]:
df_processed.to_csv('data_processed.csv', index=True)